In [6]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score, precision_recall_curve, roc_auc_score, classification_report, confusion_matrix
import joblib
import time

# --- Model Imports ---
import lightgbm as lgb
try:
    import xgboost as xgb
    has_xgb = True
except Exception:
    has_xgb = False

try:
    import optuna
    has_optuna = True
except Exception:
    has_optuna = False

try:
    from catboost import CatBoostClassifier
    has_cat = True
except Exception:
    has_cat = False

# --- 1. Load Data ---
print("Loading data...")
df = pd.read_csv("train.csv")
expected_X = [f"X{i}" for i in range(1, 19)]
cols_keep = ['company_name', 'fyear', 'status_label', 'Division', 'MajorGroup'] + expected_X
df = df[cols_keep].copy()
df = df.dropna(subset=['status_label'])
df['target'] = df['status_label'].map({'alive': 0, 'failed': 1})

# --- 2. Feature Engineering ---
print("Running feature engineering...")
df = df.sort_values(['company_name', 'fyear']).reset_index(drop=True)
for col in expected_X:
    df[f'{col}_lag1'] = df.groupby('company_name')[col].shift(1)
    df[f'{col}_diff1'] = df[col] - df[f'{col}_lag1']
    df[f'{col}_pct1'] = (df[f'{col}_diff1'] / (df[f'{col}_lag1'].replace({0: np.nan}))).replace([np.inf, -np.inf], np.nan)
    df[f'{col}_roll2_mean'] = df.groupby('company_name')[col].rolling(window=2, min_periods=1).mean().reset_index(level=0, drop=True)

pairs = [('X4', 'X1'), ('X5', 'X2'), ('X8', 'X7'), ('X10', 'X9')]
for a, b in pairs:
    df[f'{a}_div_{b}'] = df[a] / df[b].replace({0: np.nan})
    df[f'{a}_div_{b}'] = df[f'{a}_div_{b}'].replace([np.inf, -np.inf], np.nan)

for col in expected_X:
    df[f'log_{col}'] = np.log1p(df[col].clip(lower=0))

df['Division'] = df['Division'].astype(str)
df['MajorGroup'] = df['MajorGroup'].astype(str)

engineered_cols = [c for c in df.columns if c not in ['company_name', 'fyear', 'status_label', 'target'] + expected_X]
feature_cols = expected_X + [c for c in engineered_cols if c not in expected_X]

# --- 3. Setup CV and Preprocessing ---
print("Setting up cross-validation and preprocessor...")
N_SPLITS = 5
gkf = GroupKFold(n_splits=N_SPLITS)
X = df[feature_cols].copy()
y = df['target']
groups = df['company_name']

numerical_cols = [c for c in X.columns if X[c].dtype.kind in 'fi']
categorical_cols = [c for c in ['Division', 'MajorGroup'] if c in X.columns]

num_pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
cat_pipeline = Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])
preprocessor = ColumnTransformer(transformers=[('num', num_pipeline, numerical_cols), ('cat', cat_pipeline, categorical_cols)], remainder='drop')

# --- 4. Helper Functions ---
def find_best_threshold(y_true, y_proba):
    precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
    f1s = (2 * precision * recall) / (precision + recall + 1e-12)
    best_idx = np.nanargmax(f1s[:-1])
    best_t = thresholds[best_idx]
    return best_t, f1s[best_idx]

def tune_lgb(X_train, y_train, X_val, y_val, n_trials=5, random_state=42):
    if not has_optuna:
        return {'learning_rate': 0.05, 'num_leaves': 64, 'min_child_samples': 20, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0.0, 'reg_lambda': 0.0}
    
    def objective(trial):
        param = {
            'objective': 'binary',
            'metric': 'binary_logloss',
            'verbosity': -1,
            'boosting_type': 'gbdt',
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
            'num_leaves': trial.suggest_int('num_leaves', 16, 128),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.4, 1.0),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
            'random_state': random_state
        }
        model = lgb.LGBMClassifier(n_estimators=500, **param)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(50)])
        probs = model.predict_proba(X_val)[:, 1]
        _, f1 = find_best_threshold(y_val, probs)
        return 1.0 - f1

    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=random_state))
    study.optimize(objective, n_trials=n_trials, show_progress_bar=False)
    best_params = study.best_params
    best_params.pop('n_estimators', None)
    return best_params

def tune_xgb(X_train, y_train, X_val, y_val, n_trials=5, random_state=42):
    if (not has_optuna) or (not has_xgb):
        return {'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 1, 'subsample': 0.8, 'colsample_bytree': 0.8}
    
    import optuna

    def objective(trial):
        param = {
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "gamma": trial.suggest_float("gamma", 0, 5),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
            "lambda": trial.suggest_float("lambda", 1e-3, 10.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-3, 10.0, log=True),
            "random_state": random_state
        }

        model = xgb.XGBClassifier(
            n_estimators=500,
            eval_metric="logloss",
            n_jobs=1,
            tree_method="hist",
            use_label_encoder=False,
            **param
        )

        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=False
        )

        probs = model.predict_proba(X_val)[:, 1]
        _, f1 = find_best_threshold(y_val, probs)
        return 1.0 - f1

    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=random_state))
    study.optimize(objective, n_trials=n_trials, show_progress_bar=False)
    best_params = study.best_params
    best_params.pop('n_estimators', None)
    return best_params


# --- 5. Cross-Validation Training ---
fold_models = []
fold_thresholds = []
fold_scores = []
N_TRIALS = 20
print(f"Starting {N_SPLITS}-Fold GroupKFold with {N_TRIALS} Optuna trials per fold...")

fold_no = 0
for train_idx, val_idx in gkf.split(X, y, groups=groups):
    fold_no += 1
    print(f"\n--- Fold {fold_no}/{N_SPLITS} ---")
    X_tr, X_va = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]

    # Fit preprocessor on this fold's training data
    preprocessor.fit(X_tr)
    X_tr_proc = preprocessor.transform(X_tr)
    X_va_proc = preprocessor.transform(X_va)
    
    # We use the raw X_tr/X_va for CatBoost, but with NaNs filled
    X_tr_cat = X_tr.copy()
    X_va_cat = X_va.copy()
    for col in numerical_cols:
        median = X_tr_cat[col].median()
        X_tr_cat[col] = X_tr_cat[col].fillna(median)
        X_va_cat[col] = X_va_cat[col].fillna(median)
    for col in categorical_cols:
        mode = X_tr_cat[col].mode()[0]
        X_tr_cat[col] = X_tr_cat[col].fillna(mode)
        X_va_cat[col] = X_va_cat[col].fillna(mode)

    print("Tuning LightGBM...")
    lgb_params = tune_lgb(X_tr_proc, y_tr, X_va_proc, y_va, n_trials=N_TRIALS, random_state=42 + fold_no)
    lgb_model = lgb.LGBMClassifier(objective='binary', n_estimators=1000, random_state=42, **lgb_params, n_jobs=-1)
    lgb_model.set_params(class_weight='balanced')
    lgb_model.fit(X_tr_proc, y_tr, eval_set=[(X_va_proc, y_va)], callbacks=[lgb.early_stopping(50)])    
    preds_lgb = lgb_model.predict_proba(X_va_proc)[:, 1]
    preds_list = [preds_lgb]

    if has_xgb:
        print("Tuning XGBoost...")
        xgb_params = tune_xgb(X_tr_proc, y_tr, X_va_proc, y_va, n_trials=N_TRIALS, random_state=42 + fold_no)
        xgb_model = xgb.XGBClassifier(
            n_estimators=1000, 
            random_state=42, 
            use_label_encoder=False,
            eval_metric="logloss",
            **xgb_params, 
            n_jobs=-1
        )
        try:
            spw = int(y_tr.value_counts()[0] / y_tr.value_counts()[1])
            xgb_model.set_params(scale_pos_weight=spw)
        except Exception:
            pass
        xgb_model.fit(
            X_tr_proc, y_tr,
            eval_set=[(X_va_proc, y_va)],
            verbose=False
        )
        preds_xgb = xgb_model.predict_proba(X_va_proc)[:, 1]
        preds_list.append(preds_xgb)
    else:
        xgb_model = None

    if has_cat:
        try:
            print("Training CatBoost...")
            cat_model = CatBoostClassifier(verbose=0, random_state=42, n_estimators=1000, auto_class_weights='Balanced', eval_metric='F1')
            cat_model.fit(X_tr_cat, y_tr, 
                          eval_set=[(X_va_cat, y_va)],
                          cat_features=categorical_cols, 
                          early_stopping_rounds=50)
            preds_cat = cat_model.predict_proba(X_va_cat)[:, 1]
            preds_list.append(preds_cat)
        except Exception as e:
            print(f"CatBoost training error: {e}")
            cat_model = None
    else:
        cat_model = None

    avg_proba = np.mean(np.vstack(preds_list), axis=0)
    best_t, best_f1 = find_best_threshold(y_va, avg_proba)
    print(f"Fold {fold_no} best F1 (ensemble avg) = {best_f1:.4f} at threshold {best_t:.4f}")

    fold_models.append({'preprocessor': preprocessor, 'lgb': lgb_model, 'xgb': xgb_model, 'cat': cat_model})
    fold_thresholds.append(best_t)
    fold_scores.append(best_f1)

print("\nCross-validation completed.")
print("Fold F1 scores:", [round(f, 4) for f in fold_scores])
print(f"Mean CV F1: {np.mean(fold_scores):.4f}")

# --- 6. Final Model Training ---
print("\nTraining final models on all data...")
final_preprocessor = preprocessor
final_preprocessor.fit(X)
X_proc = final_preprocessor.transform(X)

# Final LGBM
final_lgb = lgb.LGBMClassifier(objective='binary', n_estimators=2000, random_state=42, class_weight='balanced', n_jobs=-1)
try:
    last_lgb = fold_models[-1]['lgb']
    for p in ['num_leaves', 'min_child_samples', 'learning_rate', 'subsample', 'colsample_bytree', 'reg_alpha', 'reg_lambda']:
        if p in last_lgb.get_params():
            final_lgb.set_params(**{p: last_lgb.get_params()[p]})
except Exception:
    pass
final_lgb.fit(X_proc, y, verbose=False)
probas = [final_lgb.predict_proba(X_proc)[:, 1]]

# Final XGB
final_xgb = None
if has_xgb:
    final_xgb = xgb.XGBClassifier(
        n_estimators=2000, 
        random_state=42, 
        use_label_encoder=False,
        eval_metric="logloss",
        n_jobs=-1
    )
    try:
        last_xgb = fold_models[-1]['xgb']
        if last_xgb is not None:
            for p in ['learning_rate', 'max_depth', 'min_child_weight', 'subsample', 'colsample_bytree', 'gamma', 'alpha', 'lambda']:
                if p in last_xgb.get_params():
                    final_xgb.set_params(**{p: last_xgb.get_params()[p]})
    except Exception:
        pass
    try:
        spw = int(y.value_counts()[0] / y.value_counts()[1])
        final_xgb.set_params(scale_pos_weight=spw)
    except Exception:
        pass
    final_xgb.fit(X_proc, y, verbose=False)
    probas.append(final_xgb.predict_proba(X_proc)[:, 1])

# Final CatBoost
final_cat = None
if has_cat:
    try:
        X_cat_final = X.copy()
        for col in numerical_cols:
            median = X_cat_final[col].median()
            X_cat_final[col] = X_cat_final[col].fillna(median)
        for col in categorical_cols:
            mode = X_cat_final[col].mode()[0]
            X_cat_final[col] = X_cat_final[col].fillna(mode)
            
        final_cat = CatBoostClassifier(verbose=0, random_state=42, n_estimators=2000, auto_class_weights='Balanced', eval_metric='F1')
        final_cat.fit(X_cat_final, y, cat_features=categorical_cols)
        probas.append(final_cat.predict_proba(X_cat_final)[:, 1])
    except Exception as e:
        print(f"Final CatBoost training failed: {e}")
        final_cat = None

# --- 7. Final Evaluation and Save ---
final_threshold = float(np.mean(fold_thresholds)) if len(fold_thresholds) > 0 else 0.5
print(f"Final threshold (mean of folds): {final_threshold:.4f}")

ensemble_proba = np.mean(np.vstack(probas), axis=0)
preds_final = (ensemble_proba >= final_threshold).astype(int)

print("\nFinal ensemble evaluation on full train (for reference):")
print(f"ROC-AUC: {roc_auc_score(y, ensemble_proba):.4f}")
print(f"F1: {f1_score(y, preds_final):.4f}")
print("Confusion matrix:")
print(confusion_matrix(y, preds_final))
print("Classification report:")
print(classification_report(y, preds_final, target_names=['alive', 'failed']))

artifact = {
    'preprocessor': final_preprocessor,
    'lgb_model': final_lgb,
    'xgb_model': final_xgb,
    'cat_model': final_cat,
    'threshold': final_threshold,
    'feature_columns': feature_cols,
    'categorical_columns_raw': categorical_cols,
    'numerical_columns_raw': numerical_cols
}
joblib.dump(artifact, 'final_ensemble.joblib')
print("\nSaved final ensemble to 'final_ensemble.joblib'")
print("Note: The final models are trained on all data. CV F1 is the best estimate of performance.")

Loading data...
Running feature engineering...
Setting up cross-validation and preprocessor...
Starting 5-Fold GroupKFold with 20 Optuna trials per fold...

--- Fold 1/5 ---


[I 2025-11-02 08:49:09,068] A new study created in memory with name: no-name-dd12fef2-bb87-4245-8a06-b07fb3a52a18


Tuning LightGBM...
Training until validation scores don't improve for 50 rounds


[I 2025-11-02 08:49:11,598] Trial 0 finished with value: 0.7400295420978991 and parameters: {'learning_rate': 0.01411527819587234, 'num_leaves': 84, 'min_child_samples': 17, 'subsample': 0.6202948099826744, 'colsample_bytree': 0.5962834334866839, 'reg_alpha': 0.5398056408643329, 'reg_lambda': 0.009881250696359372}. Best is trial 0 with value: 0.7400295420978991.


Early stopping, best iteration is:
[327]	valid_0's binary_logloss: 0.2087
Training until validation scores don't improve for 50 rounds


...

[I 2025-11-02 09:22:47,132] A new study created in memory with name: no-name-10ec2370-d747-4caa-be0b-045075191edc


Early stopping, best iteration is:
[796]	valid_0's binary_logloss: 0.282392
Tuning XGBoost...


[I 2025-11-02 09:22:56,165] Trial 0 finished with value: 0.7639296187687779 and parameters: {'max_depth': 3, 'learning_rate': 0.2750614266240631, 'subsample': 0.8643673167505531, 'colsample_bytree': 0.6757339029463507, 'gamma': 3.538025691295405, 'min_child_weight': 8, 'lambda': 0.38216192689525963, 'alpha': 0.04554056650541418}. Best is trial 0 with value: 0.7639296187687779.
[I 2025-11-02 09:23:13,448] Trial 1 finished with value: 0.7490774907753586 and parameters: {'max_depth': 8, 'learning_rate': 0.023138278510724513, 'subsample': 0.6279962156489811, 'colsample_bytree': 0.5120056770726804, 'gamma': 0.49362977424405263, 'min_child_weight': 4, 'lambda': 0.36595080871390473, 'alpha': 0.019446067650582714}. Best is trial 1 with value: 0.7490774907753586.
[I 2025-11-02 09:23:24,178] Trial 2 finished with value: 0.7722998729355579 and parameters: {'max_depth': 4, 'learning_rate': 0.2263633361598099, 'subsample': 0.635460401687385, 'colsample_bytree': 0.6367739457987344, 'gamma': 4.772063

Training CatBoost...
Fold 5 best F1 (ensemble avg) = 0.2404 at threshold 0.4263

Cross-validation completed.
Fold F1 scores: [np.float64(0.2664), np.float64(0.2632), np.float64(0.2365), np.float64(0.2793), np.float64(0.2404)]
Mean CV F1: 0.2571

Training final models on all data...


TypeError: LGBMClassifier.fit() got an unexpected keyword argument 'verbose'

In [7]:
# --- 6. Final Model Training ---
print("\nTraining final models on all data...")
final_preprocessor = preprocessor
final_preprocessor.fit(X)
X_proc = final_preprocessor.transform(X)

# Final LGBM
final_lgb = lgb.LGBMClassifier(objective='binary', n_estimators=2000, random_state=42, class_weight='balanced', n_jobs=-1, verbose=-1)
try:
    last_lgb = fold_models[-1]['lgb']
    for p in ['num_leaves', 'min_child_samples', 'learning_rate', 'subsample', 'colsample_bytree', 'reg_alpha', 'reg_lambda']:
        if p in last_lgb.get_params():
            final_lgb.set_params(**{p: last_lgb.get_params()[p]})
except Exception:
    pass
final_lgb.fit(X_proc, y)
probas = [final_lgb.predict_proba(X_proc)[:, 1]]

# Final XGB
final_xgb = None
if has_xgb:
    final_xgb = xgb.XGBClassifier(
        n_estimators=2000, 
        random_state=42, 
        use_label_encoder=False,
        eval_metric="logloss",
        n_jobs=-1
    )
    try:
        last_xgb = fold_models[-1]['xgb']
        if last_xgb is not None:
            for p in ['learning_rate', 'max_depth', 'min_child_weight', 'subsample', 'colsample_bytree', 'gamma', 'alpha', 'lambda']:
                if p in last_xgb.get_params():
                    final_xgb.set_params(**{p: last_xgb.get_params()[p]})
    except Exception:
        pass
    try:
        spw = int(y.value_counts()[0] / y.value_counts()[1])
        final_xgb.set_params(scale_pos_weight=spw)
    except Exception:
        pass
    final_xgb.fit(X_proc, y)
    probas.append(final_xgb.predict_proba(X_proc)[:, 1])

# Final CatBoost
final_cat = None
if has_cat:
    try:
        X_cat_final = X.copy()
        for col in numerical_cols:
            median = X_cat_final[col].median()
            X_cat_final[col] = X_cat_final[col].fillna(median)
        for col in categorical_cols:
            mode = X_cat_final[col].mode()[0]
            X_cat_final[col] = X_cat_final[col].fillna(mode)
            
        final_cat = CatBoostClassifier(verbose=0, random_state=42, n_estimators=2000, auto_class_weights='Balanced', eval_metric='F1')
        final_cat.fit(X_cat_final, y, cat_features=categorical_cols)
        probas.append(final_cat.predict_proba(X_cat_final)[:, 1])
    except Exception as e:
        print(f"Final CatBoost training failed: {e}")
        final_cat = None

# --- 7. Final Evaluation and Save ---
final_threshold = float(np.mean(fold_thresholds)) if len(fold_thresholds) > 0 else 0.5
print(f"Final threshold (mean of folds): {final_threshold:.4f}")

ensemble_proba = np.mean(np.vstack(probas), axis=0)
preds_final = (ensemble_proba >= final_threshold).astype(int)

print("\nFinal ensemble evaluation on full train (for reference):")
print(f"ROC-AUC: {roc_auc_score(y, ensemble_proba):.4f}")
print(f"F1: {f1_score(y, preds_final):.4f}")
print("Confusion matrix:")
print(confusion_matrix(y, preds_final))
print("Classification report:")
print(classification_report(y, preds_final, target_names=['alive', 'failed']))

artifact = {
    'preprocessor': final_preprocessor,
    'lgb_model': final_lgb,
    'xgb_model': final_xgb,
    'cat_model': final_cat,
    'threshold': final_threshold,
    'feature_columns': feature_cols,
    'categorical_columns_raw': categorical_cols,
    'numerical_columns_raw': numerical_cols
}
joblib.dump(artifact, 'final_ensemble.joblib')
print("\nSaved final ensemble to 'final_ensemble.joblib'")
print("Note: The final models are trained on all data. CV F1 is the best estimate of performance.")


Training final models on all data...
Final threshold (mean of folds): 0.5148

Final ensemble evaluation on full train (for reference):
ROC-AUC: 1.0000
F1: 0.9460
Confusion matrix:
[[58108   478]
 [    2  4201]]
Classification report:
              precision    recall  f1-score   support

       alive       1.00      0.99      1.00     58586
      failed       0.90      1.00      0.95      4203

    accuracy                           0.99     62789
   macro avg       0.95      1.00      0.97     62789
weighted avg       0.99      0.99      0.99     62789


Saved final ensemble to 'final_ensemble.joblib'
Note: The final models are trained on all data. CV F1 is the best estimate of performance.


In [9]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings("ignore")

# --- 1. Define Helper Function for Feature Engineering ---
# This function replicates the FE from your training script.

def engineer_features(df):
    """Applies all time-series and ratio features."""
    print("Applying feature engineering...")
    df_out = df.copy()
    
    # Ensure correct data types for processing
    df_out['Division'] = df_out['Division'].astype(str)
    df_out['MajorGroup'] = df_out['MajorGroup'].astype(str)

    expected_X = [f"X{i}" for i in range(1, 19)]
    
    # Sort by company and year (CRITICAL for lags/rolls)
    df_out = df_out.sort_values(['company_name', 'fyear']).reset_index(drop=True)
    
    # Create time-series features
    for col in expected_X:
        if col in df_out.columns:
            df_out[f'{col}_lag1'] = df_out.groupby('company_name')[col].shift(1)
            df_out[f'{col}_diff1'] = df_out[col] - df_out[f'{col}_lag1']
            df_out[f'{col}_pct1'] = (df_out[f'{col}_diff1'] / (df_out[f'{col}_lag1'].replace({0: np.nan}))).replace([np.inf, -np.inf], np.nan)
            df_out[f'{col}_roll2_mean'] = df_out.groupby('company_name')[col].rolling(window=2, min_periods=1).mean().reset_index(level=0, drop=True)

    # Create ratio features
    pairs = [('X4', 'X1'), ('X5', 'X2'), ('X8', 'X7'), ('X10', 'X9')]
    for a, b in pairs:
        if a in df_out.columns and b in df_out.columns:
            df_out[f'{a}_div_{b}'] = df_out[a] / df_out[b].replace({0: np.nan})
            df_out[f'{a}_div_{b}'] = df_out[f'{a}_div_{b}'].replace([np.inf, -np.inf], np.nan)

    # Create log features
    for col in expected_X:
        if col in df_out.columns:
            df_out[f'log_{col}'] = np.log1p(df_out[col].clip(lower=0))
            
    print("Feature engineering complete.")
    return df_out

# --- 2. Load Artifacts and Data ---
print("Loading 'final_ensemble.joblib' artifact...")
try:
    artifact = joblib.load('final_ensemble.joblib')
    
    # Load all necessary data
    print("Loading train.csv (for context) and test.csv (for prediction)...")
    df_train = pd.read_csv('train.csv')
    df_test = pd.read_csv('test.csv')
    
    # Store original test data for final output
    df_test_original = df_test[['company_name']].copy()
    
except FileNotFoundError as e:
    print(f"Error loading files: {e}")
    print("Please make sure 'final_ensemble.joblib', 'train.csv', and 'test.csv' are present.")
    # In a real script, you'd exit here
    # exit()
else:
    print("Artifact and data loaded.")

    # --- 3. Run Full Prediction Pipeline ---

    # Combine train/test to correctly calculate time-series features
    df_train['source'] = 'train'
    df_test['source'] = 'test'
    
    # Align columns before concat
    test_cols = df_test.columns
    # Use only columns present in test_cols from df_train
    df_full = pd.concat([df_train[test_cols], df_test], ignore_index=True)

    # Apply all feature engineering
    df_full_eng = engineer_features(df_full)
    
    # Split back into train and test (now with correct features)
    df_train_eng = df_full_eng[df_full_eng['source'] == 'train'].copy()
    df_test_eng = df_full_eng[df_full_eng['source'] == 'test'].copy()

    # Get the feature list from the artifact
    feature_cols = artifact['feature_columns']
    
    # Ensure all feature columns exist, fill with NaN if any are missing
    for col in feature_cols:
        if col not in df_test_eng.columns:
            df_test_eng[col] = np.nan
            
    X_test_eng = df_test_eng[feature_cols]

    # --- 4. Get Imputation Stats from Training Data (for CatBoost) ---
    print("Calculating training stats for CatBoost imputation...")
    # This is to correctly impute the test set based on the training set
    
    # Ensure all feature columns exist for training data as well
    for col in feature_cols:
        if col not in df_train_eng.columns:
            df_train_eng[col] = np.nan
            
    X_train_eng = df_train_eng[feature_cols]
    
    train_medians = X_train_eng[artifact['numerical_columns_raw']].median()
    train_modes = X_train_eng[artifact['categorical_columns_raw']].mode().iloc[0]

    # --- 5. Generate Predictions from Each Model ---
    print("Generating predictions...")
    
    # Load components from artifact
    preprocessor = artifact['preprocessor']
    lgb_model = artifact['lgb_model']
    xgb_model = artifact['xgb_model']
    cat_model = artifact['cat_model']
    final_threshold = artifact['threshold']
    
    probas = []

    # --- LGBM Predictions (uses preprocessor) ---
    if lgb_model:
        X_test_proc = preprocessor.transform(X_test_eng)
        preds_lgb = lgb_model.predict_proba(X_test_proc)[:, 1]
        probas.append(preds_lgb)
        print("✓ LightGBM predictions complete.")

    # --- XGBoost Predictions (uses preprocessor) ---
    if xgb_model:
        X_test_proc = preprocessor.transform(X_test_eng)
        preds_xgb = xgb_model.predict_proba(X_test_proc)[:, 1]
        probas.append(preds_xgb)
        print("✓ XGBoost predictions complete.")

    # --- CatBoost Predictions (uses manually imputed data) ---
    if cat_model:
        X_test_cat = X_test_eng.copy()
        
        # Impute using *training* stats
        X_test_cat[artifact['numerical_columns_raw']] = X_test_cat[artifact['numerical_columns_raw']].fillna(train_medians)
        X_test_cat[artifact['categorical_columns_raw']] = X_test_cat[artifact['categorical_columns_raw']].fillna(train_modes)
        
        preds_cat = cat_model.predict_proba(X_test_cat)[:, 1]
        probas.append(preds_cat)
        print("✓ CatBoost predictions complete.")

    # --- 6. Ensemble and Finalize Output ---
    if not probas:
        print("Error: No models were found in the artifact.")
    else:
        print("Ensembling predictions...")
        # Average the probabilities from all available models
        ensemble_proba = np.mean(np.vstack(probas), axis=0)
        
        # Apply the final optimized threshold
        preds_final = (ensemble_proba >= final_threshold).astype(int)
        
        # Map back to labels
        status_labels = np.where(preds_final == 1, 'failed', 'alive')

        # Create output DataFrame
        df_output = pd.DataFrame({
            'company_name': df_test_eng['company_name'],
            'status': status_labels
        })
        
        # Save to CSV
        df_output.to_csv('output.csv', index=False)
        
        print(f"\n✅ Success! Predictions saved to 'output.csv'")
        print(f"Used {len(probas)} model(s) for ensembling.")
        print(f"Applied final threshold: {final_threshold:.4f}")
        print("\nOutput file head:")
        print(df_output.head())

Loading 'final_ensemble.joblib' artifact...
Loading train.csv (for context) and test.csv (for prediction)...
Artifact and data loaded.
Applying feature engineering...
Feature engineering complete.
Calculating training stats for CatBoost imputation...
Generating predictions...
✓ LightGBM predictions complete.
✓ XGBoost predictions complete.
✓ CatBoost predictions complete.
Ensembling predictions...

✅ Success! Predictions saved to 'output.csv'
Used 3 model(s) for ensembling.
Applied final threshold: 0.5148

Output file head:
   company_name status
54       C_1000  alive
55       C_1000  alive
56       C_1000  alive
57       C_1000  alive
58       C_1000  alive
